In [1]:
!date

Tue Apr 27 11:45:40 PDT 2021


# Install packages and download data

In [7]:
import sys
COLAB = "google.colab" in sys.modules


In [2]:
if Colab:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 173 (delta 57), reused 127 (delta 25), pack-reused 0
Receiving objects: 100% (173/173), 59.25 MiB | 9.85 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [4]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6 \
    pandas==1.2.4 \
    matplotlib==3.4.1 \
    scikit-learn==0.24.1 \
    scipy==1.4.1 \
    numpy==1.20.2


In [ ]:
if COLAB:
    # download cellranger (30sec)
    !wget -O cellranger-6.0.1.tar.gz \
    -q --show-progress --progress=bar:force  \
    "https://cf.10xgenomics.com/releases/cell-exp/cellranger-6.0.1.tar.gz?Expires=1622008796&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZi4xMHhnZW5vbWljcy5jb20vcmVsZWFzZXMvY2VsbC1leHAvY2VsbHJhbmdlci02LjAuMS50YXIuZ3oiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2MjIwMDg3OTZ9fX1dfQ__&Signature=bj8nUZobiq8EHUV0VgZjB1DYafrPwOD2c5Kd8pBGurJnYTDYZhPpNQno0L3~wNrqnnWbp7ApWWkn-cLo7xp4b9r5c7KdtMGvEze6xPAtmNzblRas8Pu1RiHtZ5H7KCJEfQ9GgXuiJ-b3a-xEa2mErsm2~kda-4tuXinSBCS-cQPCT00HxmoxiuJo26vSv94QalNrd2FcTU5ABnadXtxpzv~Y6~KOjftDyIQ79Ujn5ncTFigpuLOncADaMI0qOQcOaFKG7xDPOu2xKCu12yHb~L7iH098lmjRhly4wEY8-0z2ch7pwMp-x9OdHj2HBSkXItlz1BI77E-7NfZrG0Bajw__&Key-Pair-Id=APKAI7S6A5RYOXBWRPDA" \
    2>&1
        
    !tar -xf cellranger-6.0.1.tar.gz

    
    # download cellranger reference (5min)
    !wget \
    -q --show-progress --progress=bar:force  \
    https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz \
    2>&1
    
    !tar -xf refdata-gex-GRCh38-2020-A.tar.gz


tar: This does not look like a tar archive

gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now
refdata-gex-GRCh38- 100%[===================>]  10.66G  40.6MB/s    in 4m 14s  


In [15]:
%%bash
source cellranger-6.0.1/sourceme.bash

In [3]:
if COLAB:
    # download the relevant data
    !wget -O 10xCRISPR_v3_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://cg.10xgenomics.com/samples/cell-exp/6.0.0/SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex/SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_fastqs.tar \
    2>&1
    !tar -xf 10xCRISPR_v3_fastqs.tar.gz
    
# https://cf.10xgenomics.com/samples/cell-exp/6.0.0/SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex/SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_feature_reference.csv
# Make feature barcodes
# paste -d$'\t' <(cat SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_feature_reference.csv | cut -d',' -f5) <(cat SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_feature_reference.csv | cut -d',' -f2) | tail -n +2 > feature_barcodes.txt

10xCRISPR_v3_fastqs 100%[===================>]  18.82G  28.0MB/s    in 9m 5s   


In [ ]:
!mkdir -p fastqs
!mv SC3_v3_NextGem_DI_CRISPR_A549_5K/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr/* fastqs/

In [ ]:
# trim the reads perfectly at the feature barcode position 
# for f in $(ls SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L00*_R2_001.fastq.gz); do seqtk trimfq -b 31 -e 40 $f | gzip > tmp && mv tmp $f; done

# Preprocess with kb

In [5]:
%%time
!FB="BMGP_2020/references/10xCRISPR/kite/fbcs_only/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xCRISPR/kite/fbcs_only/features.idx" && \
F2B="BMGP_2020/references/10xCRISPR/kite/fbcs_only/f2b.txt" && \
FASTA="BMGP_2020/references/10xCRISPR/kite/fbcs_only/features.fa" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-05-21 11:57:07,797]    INFO Generating mismatch FASTA at ../../../references/10xCRISPR/kite/features.fa
[2021-05-21 11:57:07,824] WARNING Detected barcodes of different lengths: 19,20
[2021-05-21 11:57:07,857]    INFO Creating transcript-to-gene mapping at ../../../references/10xCRISPR/kite/f2b.txt
[2021-05-21 11:57:07,898]    INFO Indexing ../../../references/10xCRISPR/kite/features.fa to ../../../references/10xCRISPR/kite/features.idx
CPU times: user 24.5 ms, sys: 17.5 ms, total: 42 ms
Wall time: 1.84 s


In [4]:
%%bash
FB="BMGP_2020/references/10xCRISPR/kite/fbcs_only/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xCRISPR/kite/fbcs_only/features.idx" && \
F2B="BMGP_2020/references/10xCRISPR/kite/fbcs_only/f2b.txt" && \
FASTA="BMGP_2020/references/10xCRISPR/kite/fbcs_only/features.fa" && \
OUT="kite_2" && \
FASTQS=$(FASTQDIR="fastqs_2/" &&\
         paste -d" " \
         <(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
         <(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
/usr/bin/time --output=memtime_0.log -v kb count \
--overwrite \
--h5ad \
--filter bustools \
-i $INDEX \
-g $F2B \
--workflow kite:10xFB \
-x 10xv3 \
-o $OUT \
$FASTQS

[2021-06-30 17:28:07,422]    INFO [count] Using index BMGP_2020/references/10xCRISPR/kite/fbcs_only/features.idx to generate BUS file to kite_2 from
[2021-06-30 17:28:07,422]    INFO [count]         fastqs_2/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L001_R1_001.fastq.gz
[2021-06-30 17:28:07,422]    INFO [count]         fastqs_2/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L001_R2_001.fastq.gz
[2021-06-30 17:28:07,422]    INFO [count]         fastqs_2/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L002_R1_001.fastq.gz
[2021-06-30 17:28:07,423]    INFO [count]         fastqs_2/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L002_R2_001.fastq.gz
[2021-06-30 17:28:07,423]    INFO [count]         fastqs_2/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L003_R1_001.fastq.gz
[2021-06-30 17:28:07,423]    INFO [count]         fastqs_2/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L003_R2_001.fastq.gz
[2021-06-30 17:28:07,423]    INFO [count]         fastqs_2/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_S4_L004_R1_001

# Preprocess with cellranger

In [5]:
import os
os.getcwd()

'/home/sina/projects/kite/BMGP_2020/analysis/notebooks/10xCRISPR'

In [11]:
import os
pwd = os.getcwd()
GREF=f"{pwd}/refdata-gex-GRCh38-2020-A"
FREF=f"{pwd}/BMGP_2020/references/10xCRISPR/SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_feature_reference.csv"
CELLS="5000"
FFID="SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr"
GFID="SC3_v3_NextGem_DI_CRISPR_A549_5K_gex"
FFASTQ=f"{pwd}/SC3_v3_NextGem_DI_CRISPR_A549_5K/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr"
GFASTQ=f"{pwd}/SC3_v3_NextGem_DI_CRISPR_A549_5K/SC3_v3_NextGem_DI_CRISPR_A549_5K_gex"

f = f'''[gene-expression]
reference,{GREF}
expect-cells,{CELLS}

[feature]
reference,{FREF}

[libraries]
fastq_id,fastqs,lanes,physical_library_id,feature_types,subsample_rate
{FFID},{FFASTQ},,,CRISPR Guide Capture,
{GFID},{GFASTQ},,,Gene Expression,
'''

with open("BMGP_2020/references/10xCRISPR/feat.csv", "w") as fp:
    fp.write(f)

In [12]:
!cat BMGP_2020/references/10xCRISPR/feat.csv

[gene-expression]
reference,/home/sina/projects/kite/BMGP_2020/analysis/notebooks/10xCRISPR/refdata-gex-GRCh38-2020-A
expect-cells,5000

[feature]
reference,/home/sina/projects/kite/BMGP_2020/analysis/notebooks/10xCRISPR/BMGP_2020/references/10xCRISPR/SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_feature_reference.csv

[libraries]
fastq_id,fastqs,lanes,physical_library_id,feature_types,subsample_rate
SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr,/home/sina/projects/kite/BMGP_2020/analysis/notebooks/10xCRISPR/SC3_v3_NextGem_DI_CRISPR_A549_5K/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr,,,CRISPR Guide Capture,
SC3_v3_NextGem_DI_CRISPR_A549_5K_gex,/home/sina/projects/kite/BMGP_2020/analysis/notebooks/10xCRISPR/SC3_v3_NextGem_DI_CRISPR_A549_5K/SC3_v3_NextGem_DI_CRISPR_A549_5K_gex,,,Gene Expression,


In [ ]:
# !LIB="BMGP_2020/references/10xFB_5k_pbmc_v3/5k_pbmc_protein_v3_libraries.csv" && \
# FR="BMGP_2020/references/10xFB_5k_pbmc_v3/5k_pbmc_protein_v3_feature_ref.csv" && \
# ./cellranger-6.0.1/cellranger count \
# --libraries $LIB \
# --feature-ref $FR \
# --id count \
# --nosecondary \
# --disable-ui \
# --expect-cells 5867 \
# --transcriptome refdata-gex-GRCh38-2020-A

# !LIB="../../../references/10xCRISPR/SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr_libraries.csv" && \
# FR="../../../references/10xCRISPR/SC3_v3_NextGem_DI_CRISPR_A549_5K_Multiplex_count_feature_reference.csv" && \
# ./cellranger-6.0.1/cellranger count \
# --libraries $LIB \
# --feature-ref $FR \
# --id count \
# --nosecondary \
# --disable-ui \
# --expect-cells 5867 \
# --transcriptome refdata-gex-GRCh38-2020-A

!FR="BMGP_2020/references/10xCRISPR/feat.csv" && \
/usr/bin/time --output=memtime_1.log -v ./cellranger-6.0.1/cellranger multi \
--csv $FR \
--id count \
--disable-ui

Martian Runtime - v4.0.4
Running preflight checks (please wait)...
2021-06-30 17:39:38 [runtime] (ready)           ID.count.SC_MULTI_CS.PARSE_MULTI_CONFIG
2021-06-30 17:39:38 [runtime] (run:local)       ID.count.SC_MULTI_CS.PARSE_MULTI_CONFIG.fork0.chnk0.main
2021-06-30 17:39:57 [runtime] (chunks_complete) ID.count.SC_MULTI_CS.PARSE_MULTI_CONFIG
2021-06-30 17:39:57 [runtime] (ready)           ID.count.SC_MULTI_CS.SC_MULTI_CORE.MULTI_WEBSUMMARY_BUILDER.BUILD_MULTI_GRAPH_VIEW
2021-06-30 17:39:57 [runtime] (run:local)       ID.count.SC_MULTI_CS.SC_MULTI_CORE.MULTI_WEBSUMMARY_BUILDER.BUILD_MULTI_GRAPH_VIEW.fork0.chnk0.main
2021-06-30 17:39:57 [runtime] (ready)           ID.count.SC_MULTI_CS.SC_MULTI_CORE.MULTI_CHEMISTRY_DETECTOR._GEM_WELL_CHEMISTRY_DETECTOR.DETECT_COUNT_CHEMISTRY
2021-06-30 17:39:57 [runtime] (run:local)       ID.count.SC_MULTI_CS.SC_MULTI_CORE.MULTI_CHEMISTRY_DETECTOR._GEM_WELL_CHEMISTRY_DETECTOR.DETECT_COUNT_CHEMISTRY.fork0.chnk0.main
2021-06-30 17:39:57 [runtime] (ready

In [15]:
!cat count/outs/per_sample_outs/count/metrics_summary.csv

Library or Sample,Library Type,Grouped By,Group Name,Metric Name,Metric Value
Library,CRISPR Guide Capture,Fastq ID,SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr,Number of reads,"121,241,237"
Library,CRISPR Guide Capture,Fastq ID,SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr,Number of short reads skipped,0
Library,CRISPR Guide Capture,Fastq ID,SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr,Q30 RNA read,93.0%
Library,CRISPR Guide Capture,Fastq ID,SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr,Q30 UMI,94.8%
Library,CRISPR Guide Capture,Fastq ID,SC3_v3_NextGem_DI_CRISPR_A549_5K_crispr,Q30 barcodes,94.6%
Library,CRISPR Guide Capture,Physical library ID,CGC_1,Estimated number of cells,"5,867"
Library,CRISPR Guide Capture,Physical library ID,CGC_1,Fraction guide reads,75.48%
Library,CRISPR Guide Capture,Physical library ID,CGC_1,Fraction guide reads usable,55.65%
Library,CRISPR Guide Capture,Physical library ID,CGC_1,Fraction protospacer not recognized,3.52%
Library,CRISPR Guide Capture,Physical library ID,CGC_1,Frac